In [11]:
import pandas as pd
import numpy as np

# For parsing complex GFF attributes
import re

print("Environment ready for GFF/GTF parsing")

Environment ready for GFF/GTF parsing


In [12]:
# Input file
gff_file = "/Users/emilykoehler/Downloads/NCBI-1.Mmul_10.103.fixed.gff"

# Best practice for GFF files - specify dtypes for all columns
gff_data = pd.read_csv(
    gff_file, 
    sep='\t', 
    header=None, 
    comment='#',
    names=["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"],
    dtype={
        'seqid': str,      # Chromosome names as strings
        'source': str,     # Source as strings
        'type': str,       # Feature types as strings
        'start': 'Int64',  # Integer, allows NaN (pandas nullable integer)
        'end': 'Int64',    # Integer, allows NaN
        'score': str,      # Often mixed (numbers and '.')
        'strand': str,     # '+', '-', or '.'
        'phase': str,      # '0', '1', '2', or '.'
        'attributes': str  # Always string
    }
)

# Change depending on case - Unwanted columns
gff_data = gff_data.drop(columns=['source', 'score', 'phase'])

# Change depending on case - Filter out feature types not interested in
unwanted_types = ['exon', 'match', 'CDS']
gff_data = gff_data[~gff_data['type'].isin(unwanted_types)]

# Quick check of the data
print(f"Number of rows: {len(gff_data)}")
print("\nFirst few rows:")
print(gff_data.head())

Number of rows: 128586

First few rows:
  seqid        type     start       end strand  \
0     1  pseudogene   1747947   1748243      +   
1     1  pseudogene   1818757   1819719      -   
2     1  pseudogene  26737062  26737982      +   
3     1  pseudogene  64684239  64684626      +   
4     1  pseudogene  81085685  81086088      +   

                                          attributes  
0  ID=gene-MACMULV1R-PS1002;Dbxref=GeneID:1003124...  
1  ID=gene-MACMULV1R-PS1003;Dbxref=GeneID:1003124...  
2  ID=gene-LOC701056;Dbxref=GeneID:701056;Name=LO...  
3  ID=gene-LOC718222;Dbxref=GeneID:718222;Name=LO...  
4  ID=gene-LOC705990;Dbxref=GeneID:705990;Name=LO...  


In [13]:
# Helper function to extract specific fields from GFF attributes column
def get_attribute_field(x, field, attrsep=";"):
    """Extract a specific field from GFF attributes string"""
    if pd.isna(x):  # Handle NaN values
        return None
    attributes = x.split(attrsep)
    for attr in attributes:
        if '=' in attr:
            key, value = attr.split('=', 1)
            if key.strip() == field:
                return value.strip()
    return None  # Return None if field not found

# Extract useful metadata fields from the 'attributes' column
# CHANGE DEPENDING ON CASE - Add or remove fields here depending on what's relevant
gff_data['ID'] = gff_data['attributes'].apply(lambda x: get_attribute_field(x, "ID"))
gff_data['gene'] = gff_data['attributes'].apply(lambda x: get_attribute_field(x, "gene"))
gff_data['gene_name'] = gff_data['attributes'].apply(lambda x: get_attribute_field(x, "Name"))
gff_data['gene_biotype'] = gff_data['attributes'].apply(lambda x: get_attribute_field(x, "gene_biotype"))
gff_data['transcript_id'] = gff_data['attributes'].apply(lambda x: get_attribute_field(x, "transcript_id"))
gff_data['description'] = gff_data['attributes'].apply(lambda x: get_attribute_field(x, "description"))
gff_data['Parent'] = gff_data['attributes'].apply(lambda x: get_attribute_field(x, "Parent"))

# Grouping variable - useful if grouping by gene or transcript later
# Using pandas .combine_first() as equivalent to dplyr::coalesce()
gff_data['GroupField'] = gff_data['Parent'].combine_first(gff_data['ID'])

# Reorder columns
column_order = ['GroupField', 'start', 'gene', 'gene_biotype', 'gene_name'] + \
               [col for col in gff_data.columns if col not in ['GroupField', 'start', 'gene', 'gene_biotype', 'gene_name']]
gff_data = gff_data[column_order]

# Preview data
print("Final GFF data preview:")
print(gff_data.head())
print(f"\nData shape: {gff_data.shape}")

Final GFF data preview:
              GroupField     start              gene gene_biotype  \
0  gene-MACMULV1R-PS1002   1747947  MACMULV1R-PS1002   pseudogene   
1  gene-MACMULV1R-PS1003   1818757  MACMULV1R-PS1003   pseudogene   
2         gene-LOC701056  26737062         LOC701056   pseudogene   
3         gene-LOC718222  64684239         LOC718222   pseudogene   
4         gene-LOC705990  81085685         LOC705990   pseudogene   

          gene_name seqid        type       end strand  \
0  MACMULV1R-PS1002     1  pseudogene   1748243      +   
1  MACMULV1R-PS1003     1  pseudogene   1819719      -   
2         LOC701056     1  pseudogene  26737982      +   
3         LOC718222     1  pseudogene  64684626      +   
4         LOC705990     1  pseudogene  81086088      +   

                                          attributes                     ID  \
0  ID=gene-MACMULV1R-PS1002;Dbxref=GeneID:1003124...  gene-MACMULV1R-PS1002   
1  ID=gene-MACMULV1R-PS1003;Dbxref=GeneID:1003124...  

In [14]:
# Sanity check to look for successful extraction
print("Extracted fields summary:")
for field in ['ID', 'gene', 'gene_name', 'gene_biotype', 'Parent']:
    non_null_count = gff_data[field].notna().sum()
    print(f"{field}: {non_null_count} non-null values")

Extracted fields summary:
ID: 128586 non-null values
gene: 128561 non-null values
gene_name: 127002 non-null values
gene_biotype: 40283 non-null values
Parent: 88278 non-null values
